In [1]:
import sys
sys.path.append('/workspaces/codespaces-blank/project/stock/national/yj')

import api
import get_symbol as gsym
import datetime
import time
import numpy as np
from multiprocessing import Process, Manager, Queue

In [2]:
api.open_config_file(False)

In [5]:
def refresh_symbol():    # 종목 리스트 갱신
    gsym.open_config_file(False)    # 종목 리스트 파일 열기
    gsym.initialize_symbol()    # 종목 리스트 초기화
    symbol_list = gsym.get_entire_symbol()    # 전체 종목 리스트 가져오기
    gsym.get_symbol_by_distance(symbol_list)    # 이동 평균선 간격에 따른 종목 선별
    final_symbol = gsym.get_final_symbol_list()    # 최종 선별된 종목 리스트 가져오기
    gsym.write_symbol_file(final_symbol)    # 최종 선별된 종목 리스트 파일에 쓰기

def make_request(term_of_bar: str="3", term_of_line: str="5"):
    f_symbol = open("/workspaces/codespaces-blank/project/stock/national/yj/data/symbol.txt", "r")
    symbol_list = f_symbol.readlines()
    f_symbol.close()
    for idx, symbol in enumerate(symbol_list):
        symbol_list[idx] = symbol.strip()
    
    f_request = open("/workspaces/codespaces-blank/project/stock/national/yj/data/request.txt", "w")
    f_request.write("")
    for symbol in symbol_list:
        req: str = symbol+"\n"+term_of_bar+"\n"
        f_request.write(req)
    f_request.write(term_of_line)
    f_request.close()

    return symbol_list

class Status:
    max_budget: float = api.get_balance(msg=False)*0.7

    def __init__(self, code: str="005930", file_name: str="0", short_ratio: float=0.4, long_ratio: float=0.4):
        self.code: str = code
        self.file_name: str = file_name

        self.short_ratio: float = short_ratio
        self.long_ratio: float = long_ratio

        self.decision: bool = False
        self.long: bool = False
        self.short: bool = False

        self.long_line: float = 0.0
        self.short_line: float = 0.0
        self.qty_score: float = 0.0

        self.budget: float = 0.0

        self.bought: bool = False
        self.bought_qty: int = 0
        self.qty_to_buy: int = 0
        self.qty_to_sell: int = self.bought_qty


def get_trade_line(attributes, status: Status, trading: str="short"):
    if trading == "short":
        f_line = open("/workspaces/codespaces-blank/project/stock/national/yj/line/support"+status.file_name+".txt", "r")
        trading_ratio: float = status.short_ratio
    else:
        f_line = open("/workspaces/codespaces-blank/project/stock/national/yj/line/resistant"+status.file_name+".txt", "r")
        trading_ratio: float = status.long_ratio
    
    line_list = f_line.readlines()
    for idx, line in enumerate(line_list):
        line_list[idx] = line.strip()
    f_line.close()
    if "" in line_list:
        line_list.remove("")
    
    if len(line_list) >= 3:
        basis_line_list = list()
        for line in line_list:
            basis_line_list.append(float(line))
        
        basis_line: float = np.mean(basis_line_list)
        trading_line: float = basis_line*trading_ratio

        if trading == "short":
            attributes.put("short_line", trading_line)
        else:
            attributes.put("long_line", trading_line)

        return trading_line
    else:
        return 0.0

def get_decision(attributes, status: Status):
    attributes = {
        'decision': False,
        'long': False,
        'short': False,
        'long_line': 0.0,
        'short_line': 0.0,
        'qty_score': 0.0,
        'budget': 0.0,
        'qty_to_buy': 0}
    
    flag = get_trade_line(attributes, status, "short")
    get_trade_line(attributes, status, "long")

    if flag != 0.0:
        attributes.put("decision", True)
        price = api.get_current_price(status.code)
        if price <= attributes.get("short_line"):
            attributes.put("short", True)
        elif price >= attributes.get("long_line"):
            attributes.put("long", True)
    else:
        attributes.put("decision", False)

def get_qty_score(status: Status):
    price = api.get_current_price(status.code)
    status.qty_score = price/status.long_line

def get_qty_to_buy(status_li: list=[]):
    Status.max_budget = api.get_balance(msg=False)*0.7
    num_of_status: int = len(status_li)
    status_li.sort(key=lambda x: x.qty_score, reverse=True)
    for idx, status in enumerate(status_li):
        rank = idx+1
        status.budget = Status.max_budget*(num_of_status+1-rank)/(num_of_status*(num_of_status+1)/2)
    
    for status in status_li:
        price = api.get_current_price(status.code)
        status.qty_to_buy = int(status.budget/price)

In [6]:
term_of_bar = "3"
term_of_line = "3"
symbol_list = make_request(term_of_bar, term_of_line)

status_instances = [Status(symbol_list[i], str(i), 0.4, 0.4) for i in range(len(symbol_list))]

attributes_queues = [Queue() for _ in range(len(symbol_list))]

while True:
    procs = []
    for idx, (status_instance, attributes_queue) in enumerate(zip(status_instances, attributes_queues)):
        proc = Process(target=get_decision, args=(attributes_queue, status_instance))
        procs.append(proc)
        proc.start()
    for proc in procs:
        proc.join()

    for idx, (status_instance, attributes_queue) in enumerate(zip(status_instances, attributes_queues)):
        status_instance.decision = attributes_queue.get(('decision'))
        status_instance.long = attributes_queue.get(('long', False))
        status_instance.short = attributes_queue.get(('short', False))
        status_instance.long_line = attributes_queue.get(('long_line', 0.0))
        status_instance.short_line = attributes_queue.get(('short_line', 0.0))
        status_instance.qty_score = attributes_queue.get(('qty_score', 0.0))
        status_instance.budget = attributes_queue.get(('budget', 0.0))
        status_instance.qty_to_buy = attributes_queue.get(('qty_to_buy', 0))

        print(status_instance.decision, status_instance.long, status_instance.short, status_instance.long_line, status_instance.short_line, status_instance.qty_score, status_instance.budget, status_instance.qty_to_buy)

    for idx, status_instance in enumerate(status_instances):
        if status_instance.decision:
            if status_instance.long:
                #api.buy(symbol_list[idx], str(status_instance.qty_to_buy))
                status_instance.bought = True
                status_instance.bought_qty += status_instance.qty_to_buy
            elif status_instance.short and status_instance.bought:
                #api.sell(symbol_list[idx], str(status_instance.qty_to_sell))
                status_instance.bought = False
                status_instance.bought_qty = 0
                #refresh_symbol()
                #make_request(term_of_bar, term_of_line)
            #time.sleep(1)

False False False 0.0 0.0 0.0 0.0 0
False False False 0.0 0.0 0.0 0.0 0
False False False 0.0 0.0 0.0 0.0 0


AttributeError: 'str' object has no attribute 'get'